In [2]:
# import libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
# Create a Path
path = r'/Users/wiltonngo/InstaCart Basket Analysis'

In [4]:
# import orders_products_combined
df= pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'customers 2.csv'))

In [5]:
# import orders_products_combined
ords_prods_merged = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ord_prods_merged.pkl'))

In [6]:
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# Data Wrangling

In [7]:
# renaming columns

df.rename(columns = {'First Name' : 'first_name', 'Surnam' : 'last_name', 'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'n_dependants' : 'dependants_number', 'fam_status' : 'family_status'}, inplace = True)

### I'm not droping any columns because I don't know what I need. What columns are useful. 

# Data Quality and Consistenct Checks

In [8]:
df.shape

(206209, 10)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            206209 non-null  int64 
 1   first_name         194950 non-null  object
 2   last_name          206209 non-null  object
 3   gender             206209 non-null  object
 4   state              206209 non-null  object
 5   age                206209 non-null  int64 
 6   date_joined        206209 non-null  object
 7   dependants_number  206209 non-null  int64 
 8   family_status      206209 non-null  object
 9   income             206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


### There are missing first names with only 194950 entries. There should be 206209 entries

In [10]:
df.describe()

,user_id,age,dependants_number,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [11]:
# Check for Mix Types
for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)
    print (len(df[weird]))

first_name
11259


# Missing Values

In [12]:
df.isnull().sum()

user_id                  0
first_name           11259
last_name                0
gender                   0
state                    0
age                      0
date_joined              0
dependants_number        0
family_status            0
income                   0
dtype: int64

In [13]:
# View Missing Values
df[df['first_name'].isnull() == True]

,user_id,first_name,last_name,gender,state,age,date_joined,dependants_number,family_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


In [14]:
# Check to see Any Duplicates
df[df.duplicated()]

,user_id,first_name,last_name,gender,state,age,date_joined,dependants_number,family_status,income


In [15]:
#Drop Columns that isn't neceassry for InstaCart Data
df_customers = df.drop(columns = ['first_name', 'last_name'])

In [16]:
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)
  else:
    print ('No mixed-type data found in', col)

No mixed-type data found in user_id
No mixed-type data found in gender
No mixed-type data found in state
No mixed-type data found in age
No mixed-type data found in date_joined
No mixed-type data found in dependants_number
No mixed-type data found in family_status
No mixed-type data found in income


In [17]:
df_customers.isnull().sum()

user_id              0
gender               0
state                0
age                  0
date_joined          0
dependants_number    0
family_status        0
income               0
dtype: int64

# Combine Data

In [18]:
ords_prods_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 23 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int64   
 1   user_id                int64   
 2   order_number           int64   
 3   order_day_of_week      int64   
 4   order_hour_of_day      int64   
 5   days_since_last_order  float64 
 6   product_id             int64   
 7   add_to_cart_order      int64   
 8   reordered              int64   
 9   product_name           object  
 10  aisle_id               int64   
 11  department_id          int64   
 12  prices                 float64 
 13  _merge                 category
 14  price_range_loc        object  
 15  Busiest Day            object  
 16  Busiest Period of Day  object  
 17  max_order              int64   
 18  loyatly_flags          object  
 19  avg_price              float64 
 20  spending_flag          object  
 21  Meidan_Freq            float6

In [19]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            206209 non-null  int64 
 1   gender             206209 non-null  object
 2   state              206209 non-null  object
 3   age                206209 non-null  int64 
 4   date_joined        206209 non-null  object
 5   dependants_number  206209 non-null  int64 
 6   family_status      206209 non-null  object
 7   income             206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


In [22]:
# merging dataframes

dataframe_combined = ords_prods_merged.merge(df_customers, on = 'user_id')

In [23]:
dataframe_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 30 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int64   
 1   user_id                int64   
 2   order_number           int64   
 3   order_day_of_week      int64   
 4   order_hour_of_day      int64   
 5   days_since_last_order  float64 
 6   product_id             int64   
 7   add_to_cart_order      int64   
 8   reordered              int64   
 9   product_name           object  
 10  aisle_id               int64   
 11  department_id          int64   
 12  prices                 float64 
 13  _merge                 category
 14  price_range_loc        object  
 15  Busiest Day            object  
 16  Busiest Period of Day  object  
 17  max_order              int64   
 18  loyatly_flags          object  
 19  avg_price              float64 
 20  spending_flag          object  
 21  Meidan_Freq            float6

In [ ]:
dataframe_combined

In [25]:
# Exporting DataFrame
dataframe_combined.to_pickle(os.path.join(path,'Data','Prepared Data', 'dataframe_combined.pkl'))

In [1]:
dataframe_combined['department_id'].value_counts()

NameError: name 'dataframe_combined' is not defined